In [84]:
import pandas as pd
import numpy as np
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
from sklearn.model_selection import train_test_split

from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout
# import keras_tuner

In [85]:
merged_data_path = '../data/merged_data.jsonl'
data = pd.read_json(merged_data_path, lines=True)

# data = data.drop(
#     columns=["release_date", "key", "loudness",
#              "explicit", "popularity", "duration_ms", "danceability", "energy", "speechiness",
#              "acousticness", "instrumentalness", "liveness", "valence", "tempo"])

data = data.drop(
    columns=["release_date", "name"])

data.head(5)

,timestamp,user_id,track_id,session_id,track_name,popularity,duration_ms,explicit,id_artist,danceability,...,favourite_genres,premium_user,genres,skipped,number_of_matching_genres,month,day_of_week,hour_of_day,genres_with_favourite_genres,common_genres
0,2023-01-03 05:09:55.000,101,2PmGtDUyJIpYBEtI1hQIVp,124,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,0.491,...,"[permanent wave, mandopop, funk]",False,"[album rock, art rock, classic rock, folk rock...",False,0,1,1,5,"[album rock, art rock, classic rock, folk rock...","[classic rock, folk rock, protopunk, permanent..."
1,2023-02-08 21:41:02.507,103,2PmGtDUyJIpYBEtI1hQIVp,151,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,0.491,...,"[filmi, regional mexican, folk]",False,"[album rock, art rock, classic rock, folk rock...",False,0,2,2,21,"[album rock, art rock, classic rock, folk rock...","[classic rock, folk rock, protopunk, psychedel..."
2,2023-02-12 01:17:14.946,132,2PmGtDUyJIpYBEtI1hQIVp,544,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,0.491,...,"[psychedelic rock, country rock, rock en espanol]",True,"[album rock, art rock, classic rock, folk rock...",False,1,2,6,1,"[album rock, art rock, classic rock, folk rock...","[classic rock, folk rock, protopunk, country r..."
3,2023-01-03 03:59:59.738,132,5yxYokipsWlpDCt4Th4VVc,534,Solid Gold Easy Action,35,140067,0,3dBVyJ7JuOMt4GE9607Qin,0.449,...,"[psychedelic rock, country rock, rock en espanol]",True,"[album rock, art rock, classic rock, folk rock...",False,1,1,1,3,"[album rock, art rock, classic rock, folk rock...","[classic rock, folk rock, protopunk, country r..."
4,2023-03-07 14:28:25.702,132,5yxYokipsWlpDCt4Th4VVc,547,Solid Gold Easy Action,35,140067,0,3dBVyJ7JuOMt4GE9607Qin,0.449,...,"[psychedelic rock, country rock, rock en espanol]",True,"[album rock, art rock, classic rock, folk rock...",False,1,3,1,14,"[album rock, art rock, classic rock, folk rock...","[classic rock, folk rock, protopunk, country r..."


In [86]:
unique_favourite_genres = set()
for genres in data['favourite_genres']:
  for genre in genres:
    unique_favourite_genres.add(genre)

unique_genres = set()
for genres in data['genres']:
  for genre in genres:
    unique_genres.add(genre)

# show all common genres between all favourite_genres and genres
print("unique_favourite_genres", len(unique_favourite_genres))
print("unique_genres", len(unique_genres))
common_genres = unique_favourite_genres.intersection(unique_genres)
print("common genres", common_genres)

# remove genres that are not in favourite_genres
data['genres'] = data['genres'].apply(
    lambda x: [genre for genre in x if genre in unique_favourite_genres])

data.head()

# TODO remove ??? moze niepotrzebne

unique_favourite_genres 46
unique_genres 1766
common genres {'tropical', 'quiet storm', 'lounge', 'brill building pop', 'new wave pop', 'album rock', 'j-pop', 'hoerspiel', 'mandopop', 'soft rock', 'permanent wave', 'country rock', 'pop rock', 'filmi', 'dance pop', 'mpb', 'blues rock', 'new romantic', 'mellow gold', 'art rock', 'europop', 'vocal jazz', 'hard rock', 'singer-songwriter', 'rock', 'latin pop', 'pop', 'new wave', 'metal', 'rock en espanol', 'psychedelic rock', 'motown', 'alternative rock', 'regional mexican', 'ranchera', 'soul', 'latin rock', 'latin', 'turkish pop', 'c-pop', 'alternative metal', 'classic rock', 'funk', 'latin alternative', 'folk', 'adult standards'}


,timestamp,user_id,track_id,session_id,track_name,popularity,duration_ms,explicit,id_artist,danceability,...,favourite_genres,premium_user,genres,skipped,number_of_matching_genres,month,day_of_week,hour_of_day,genres_with_favourite_genres,common_genres
0,2023-01-03 05:09:55.000,101,2PmGtDUyJIpYBEtI1hQIVp,124,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,0.491,...,"[permanent wave, mandopop, funk]",False,"[album rock, art rock, classic rock, folk rock...",False,0,1,1,5,"[album rock, art rock, classic rock, folk rock...","[classic rock, folk rock, protopunk, permanent..."
1,2023-02-08 21:41:02.507,103,2PmGtDUyJIpYBEtI1hQIVp,151,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,0.491,...,"[filmi, regional mexican, folk]",False,"[album rock, art rock, classic rock, folk rock...",False,0,2,2,21,"[album rock, art rock, classic rock, folk rock...","[classic rock, folk rock, protopunk, psychedel..."
2,2023-02-12 01:17:14.946,132,2PmGtDUyJIpYBEtI1hQIVp,544,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,0.491,...,"[psychedelic rock, country rock, rock en espanol]",True,"[album rock, art rock, classic rock, folk rock...",False,1,2,6,1,"[album rock, art rock, classic rock, folk rock...","[classic rock, folk rock, protopunk, country r..."
3,2023-01-03 03:59:59.738,132,5yxYokipsWlpDCt4Th4VVc,534,Solid Gold Easy Action,35,140067,0,3dBVyJ7JuOMt4GE9607Qin,0.449,...,"[psychedelic rock, country rock, rock en espanol]",True,"[album rock, art rock, classic rock, folk rock...",False,1,1,1,3,"[album rock, art rock, classic rock, folk rock...","[classic rock, folk rock, protopunk, country r..."
4,2023-03-07 14:28:25.702,132,5yxYokipsWlpDCt4Th4VVc,547,Solid Gold Easy Action,35,140067,0,3dBVyJ7JuOMt4GE9607Qin,0.449,...,"[psychedelic rock, country rock, rock en espanol]",True,"[album rock, art rock, classic rock, folk rock...",False,1,3,1,14,"[album rock, art rock, classic rock, folk rock...","[classic rock, folk rock, protopunk, country r..."


In [87]:
# balance set

skipped_data = data[data["skipped"] == 1]
not_skipped_data = data[data["skipped"] == 0]

# Split the dataset into subsets based on class labels
print("skipped_data", skipped_data.shape)
print("not_skipped_data", not_skipped_data.shape)

# Calculate the minimum number of samples among all classes
min_samples = min(len(skipped_data), len(not_skipped_data))

# Randomly select samples from each class subset to match the minimum number of samples
skipped_data_balanced = skipped_data.sample(n=min_samples, random_state=42)
not_skipped_data_balanced = not_skipped_data.sample(n=min_samples, random_state=42)

# Merge the balanced subsets to create the final balanced dataset
balanced_data = pd.concat([skipped_data_balanced, not_skipped_data_balanced], axis=0)

# Shuffle the balanced dataset
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)
balanced_data.head()

skipped_data (3621, 32)
not_skipped_data (6260, 32)


,timestamp,user_id,track_id,session_id,track_name,popularity,duration_ms,explicit,id_artist,danceability,...,favourite_genres,premium_user,genres,skipped,number_of_matching_genres,month,day_of_week,hour_of_day,genres_with_favourite_genres,common_genres
0,2023-03-23 18:05:18.096,147,0kVB6PeqBbN2HhwJdWumeZ,732,The Wall Street Shuffle,40,236373,0,6i6WlGzQtXtz7GcC5H5st5,0.486,...,"[soft rock, hoerspiel, ranchera]",True,"[album rock, art rock, classic rock, classic u...",True,1,3,3,18,"[album rock, art rock, classic rock, classic u...","[classic uk pop, classic rock, folk rock, rock..."
1,2023-01-22 05:30:03.002,141,4pJKzul9oXW1lBQmOpf96m,646,Deep Dark Dungeon,38,129133,0,06nsZ3qSOYZ2hPVIMcr1IN,0.740,...,"[tropical, permanent wave, mellow gold]",False,"[album rock, blues, blues rock, classic rock, ...",False,1,1,6,5,"[album rock, blues, blues rock, classic rock, ...","[blues rock, classic rock, blues, country rock..."
2,2023-02-23 05:54:20.102,125,7t6zMZSO6DfCeE4IcinxUJ,443,Chala Chala Navbala,0,193320,0,61JrslREXq98hurYL2hYoc,0.488,...,"[classic rock, regional mexican, pop rock]",False,"[classic bollywood, desi pop, filmi, sufi]",False,0,2,3,5,"[classic bollywood, desi pop, filmi, sufi, cla...","[classic bollywood, classic rock, pop rock, fi..."
3,2023-02-26 04:48:49.355,149,62UVjAjo8ra1PVSeHfnxk3,754,I Love Paris,21,353200,0,3XOVABzceOUTbR3iEz0ImO,0.601,...,"[psychedelic rock, new wave pop, alternative r...",False,"[bossa nova, cool jazz, jazz, jazz funk, jazz ...",True,0,2,6,4,"[bossa nova, cool jazz, jazz, jazz funk, jazz ...","[jazz vibraphone, latin jazz, jazz, lounge, ja..."
4,2023-01-31 09:21:51.897,134,7DDfGy1cstvJGETyYYnfig,559,The One to Sing the Blues,30,187213,0,1DFr97A9HnbV3SKTJFu62M,0.347,...,"[hoerspiel, quiet storm, brill building pop]",True,"[album rock, alternative metal, hard rock, met...",True,0,1,1,9,"[album rock, alternative metal, hard rock, met...","[metal, brill building pop, hard rock, alterna..."


In [88]:
def normalize(data):
  return (data - np.min(data)) / (np.max(data) - np.min(data))

# # Combine genres and favourite_genres
all_genres = list(balanced_data['favourite_genres'] + balanced_data['genres'])

# # One-hot encode the genres
mlb = MultiLabelBinarizer()
mlb.fit(all_genres)

# encoded_all_genres = mlb.fit_transform(all_genres)

# TODO moze trzeba inaczej enkodowac?
#encoded_genres_with_favourite_genres = mlb.fit_transform(balanced_data['genres_with_favourite_genres'])
# encoded_common_genres = mlb.fit_transform(balanced_data['common_genres'])

# TODO enkodowac genres_with_favourite_genres naraz

# Split encoded_genres into genres and favourite_genres
# encoded_favourite_genres = encoded_all_genres[:, :len(balanced_data['favourite_genres'][0])]
# encoded_genres = encoded_all_genres[:, len(balanced_data['favourite_genres'][0]):]

encoded_favourite_genres = mlb.transform(balanced_data['favourite_genres'])
encoded_genres = mlb.transform(balanced_data['genres'])
# encoded_common_genres = mlb.transform(data['common_genres'])

timestamp_normalized = balanced_data['timestamp'].values.reshape(-1, 1)
timestamp_normalized = normalize(timestamp_normalized)

popularity_normalized = balanced_data['popularity'].values.reshape(-1, 1)
popularity_normalized = normalize(popularity_normalized)

tempo_normalized = balanced_data['tempo'].values.reshape(-1, 1)
tempo_normalized = normalize(tempo_normalized)

duration_ms_normalized = balanced_data['duration_ms'].values.reshape(-1, 1)
duration_ms_normalized = normalize(duration_ms_normalized)

loudness_normalized = balanced_data['loudness'].values.reshape(-1, 1)
loudness_normalized = normalize(loudness_normalized)

danceability_normalized = balanced_data['danceability'].values.reshape(-1, 1)
danceability_normalized = normalize(danceability_normalized)

energy_normalized = balanced_data['energy'].values.reshape(-1, 1)
energy_normalized = normalize(energy_normalized)

number_of_matching_genres_normalized = balanced_data['number_of_matching_genres'].values.reshape(-1, 1)
number_of_matching_genres_normalized = normalize(number_of_matching_genres_normalized)

hour_of_day_normalized = balanced_data['hour_of_day'].values.reshape(-1, 1)
hour_of_day_normalized = normalize(hour_of_day_normalized)

day_of_week_normalized = balanced_data['day_of_week'].values.reshape(-1, 1)
day_of_week_normalized = normalize(day_of_week_normalized)

premium_user_normalized = balanced_data['premium_user'].astype(int).values.reshape(-1, 1)
premium_user_normalized = normalize(premium_user_normalized)

# create data frame from data genres, data favourite_genres, encoded genres, encoded favourite_genres
df = pd.DataFrame(
    data={
          'genres': balanced_data['genres'],
          'favourite_genres': balanced_data['favourite_genres'],
          'skipped': balanced_data['skipped']})

df.head(50)

,genres,favourite_genres,skipped
0,"[album rock, art rock, classic rock, classic u...","[soft rock, hoerspiel, ranchera]",True
1,"[album rock, blues, blues rock, classic rock, ...","[tropical, permanent wave, mellow gold]",False
2,"[classic bollywood, desi pop, filmi, sufi]","[classic rock, regional mexican, pop rock]",False
3,"[bossa nova, cool jazz, jazz, jazz funk, jazz ...","[psychedelic rock, new wave pop, alternative r...",True
4,"[album rock, alternative metal, hard rock, met...","[hoerspiel, quiet storm, brill building pop]",True
5,"[kentucky hip hop, pop, pop rap, r&b, rap, trap]","[hoerspiel, j-pop, filmi]",False
6,"[album rock, art rock, classic rock, folk rock...","[soft rock, hoerspiel, ranchera]",True
7,"[brooklyn indie, indie pop]","[c-pop, latin pop, ranchera]",True
8,"[album rock, canadian singer-songwriter, class...","[filmi, regional mexican, folk]",False
9,"[contemporary country, country, country dawn, ...","[hoerspiel, regional mexican, folk]",False


In [89]:
# rearrange X in such way, that it containts equal number of skipped=True and skipped=False


In [90]:
# TODO pewnie trzeba zbalansowac set najpierw 

# # create pandas dataframe from encoded_genres, encoded_favourite_genres and skipped
# new_data = np.concatenate([encoded_genres, encoded_favourite_genres, data['skipped'].astype(int).values], axis=1)

# skipped_data = new_data[new_data["skipped"] == 1]
# not_skipped_data = new_data[new_data["skipped"] == 0]

# # Split the dataset into subsets based on class labels
# print("skipped_data", skipped_data.shape)
# print("not_skipped_data", not_skipped_data.shape)

# # Calculate the minimum number of samples among all classes
# min_samples = min(len(skipped_data), len(not_skipped_data))

# # Randomly select samples from each class subset to match the minimum number of samples
# skipped_data_balanced = skipped_data.sample(n=min_samples, random_state=42)
# not_skipped_data_balanced = not_skipped_data.sample(n=min_samples, random_state=42)

# # Merge the balanced subsets to create the final balanced dataset
# balanced_data = pd.concat([skipped_data_balanced, not_skipped_data_balanced], axis=0)

# # Shuffle the balanced dataset
# balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)
# print(balanced_data.head())


# Concatenate the one-hot encoded columns
X = np.concatenate([encoded_genres, encoded_favourite_genres, tempo_normalized, duration_ms_normalized, loudness_normalized], axis=1)
# X = np.concatenate([encoded_track_name], axis=1)
# X = np.concatenate(
#     [popularity_normalized, duration_ms_normalized, danceability_normalized, energy_normalized,
#      number_of_matching_genres_normalized], axis=1)

#
# Extract the labels
y = balanced_data['skipped'].astype(int).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

# count skipped and not skipped songs in X_train
print("Y_train not skipped", np.count_nonzero(y_test == 0))
print("Y_train skipped", np.count_nonzero(y_test == 1))
print("Y_train skipped %", np.count_nonzero(y_test == 1) / len(y_test))


Y_train not skipped 686
Y_train skipped 763
Y_train skipped % 0.5265700483091788


In [91]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(3000 * 2, activation='relu', input_shape=(X_train.shape[1],)))
  model.add(Dropout(0.5))
  model.add(Dense(3000, activation='relu'))
  model.add(Dropout(0.5))
  # model.add(Dense(1000, activation='relu'))
  # model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  # model.compile(
  #     optimizer=keras.optimizers.Adam(hp.Choice('learning_date', values=[0.5, 0.1, 0.01])),
  #     loss='binary_crossentropy', metrics=['accuracy'])
  model.compile(
      optimizer=Adam(),
      loss='binary_crossentropy', metrics=['accuracy'])
  return model


In [92]:
epochs = 50
batch_size = 64

model = build_model(None)

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=epochs,
    batch_size=batch_size,
    verbose=2
)

# tuner = keras_tuner.tuners.Hyperband(
#     build_model,
#     objective='val_accuracy',
#     max_epochs=50,
#     max_trials=10,
#     executions_per_trial=2,
#     directory='my_dir')


Epoch 1/50
91/91 - 2s - loss: 0.6503 - accuracy: 0.6273 - val_loss: 0.6001 - val_accuracy: 0.6756 - 2s/epoch - 27ms/step
Epoch 2/50
91/91 - 2s - loss: 0.5690 - accuracy: 0.7027 - val_loss: 0.6162 - val_accuracy: 0.6812 - 2s/epoch - 19ms/step
Epoch 3/50
91/91 - 2s - loss: 0.5248 - accuracy: 0.7352 - val_loss: 0.6345 - val_accuracy: 0.6680 - 2s/epoch - 19ms/step
Epoch 4/50
91/91 - 2s - loss: 0.4713 - accuracy: 0.7692 - val_loss: 0.7426 - val_accuracy: 0.6522 - 2s/epoch - 18ms/step
Epoch 5/50
91/91 - 2s - loss: 0.4241 - accuracy: 0.7880 - val_loss: 0.7370 - val_accuracy: 0.6549 - 2s/epoch - 19ms/step
Epoch 6/50
91/91 - 2s - loss: 0.3865 - accuracy: 0.8082 - val_loss: 0.7964 - val_accuracy: 0.6598 - 2s/epoch - 19ms/step
Epoch 7/50
91/91 - 2s - loss: 0.3611 - accuracy: 0.8220 - val_loss: 0.8606 - val_accuracy: 0.6467 - 2s/epoch - 18ms/step
Epoch 8/50
91/91 - 2s - loss: 0.3269 - accuracy: 0.8362 - val_loss: 0.9530 - val_accuracy: 0.6259 - 2s/epoch - 18ms/step
Epoch 9/50
91/91 - 2s - loss: 0.

KeyboardInterrupt: 

In [ ]:
# tuner.search(
#     (X_train, y_train),
#     validation_data=(X_test, y_test),
# )

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print("Test set accuracy:", accuracy)

print("TEST")
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)
print("Accuracy:", accuracy_score(y_test, y_pred_classes))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred_classes))
print("Classification report:\n", classification_report(y_test, y_pred_classes))

print("TRAIN")
y_pred = model.predict(X_train)
y_pred_classes = (y_pred > 0.5).astype(int)
print("Accuracy:", accuracy_score(y_train, y_pred_classes))
print("Confusion matrix:\n", confusion_matrix(y_train, y_pred_classes))
print("Classification report:\n", classification_report(y_train, y_pred_classes))

46/46 - 0s - loss: 2.0469 - accuracy: 0.6335 - 275ms/epoch - 6ms/step
Test set accuracy: 0.6335403919219971
TEST
46/46 [==============================] - 0s 3ms/step
Accuracy: 0.6335403726708074
Confusion matrix:
 [[456 257]
 [274 462]]
Classification report:
               precision    recall  f1-score   support

           0       0.62      0.64      0.63       713
           1       0.64      0.63      0.64       736

    accuracy                           0.63      1449
   macro avg       0.63      0.63      0.63      1449
weighted avg       0.63      0.63      0.63      1449

TRAIN
182/182 [==============================] - 1s 3ms/step
Accuracy: 0.919903331607112
Confusion matrix:
 [[2626  282]
 [ 182 2703]]
Classification report:
               precision    recall  f1-score   support

           0       0.94      0.90      0.92      2908
           1       0.91      0.94      0.92      2885

    accuracy                           0.92      5793
   macro avg       0.92      0.92  

In [ ]:
# # Make predictions on the test set
# y_pred = model.predict(X_test)
# y_pred_classes = (y_pred > 0.5).astype(int)
#
# # Print the predicted and actual labels
# print("Predicted labels:", y_pred_classes.flatten())
# print("Actual labels:", y_test)

In [ ]:
# # test with new data
#
# new_data = [
#   (["dominican pop", "merengue", "merengue tipico", "tropical"],
#    ["blues rock", "country rock", "lounge"])
# ]
#
# labels = [
#
# ]
#
# new_df = pd.DataFrame(new_data, columns=["genres", "favourite_genres"])
#
# # Combine genres and favourite_genres
# all_new_genres = list(new_df['genres'] + new_df['favourite_genres'])
#
# # One-hot encode the genres using the previously fit MultiLabelBinarizer (mlb)
# encoded_new_genres = mlb.transform(all_new_genres)
#
# # Split encoded_new_genres into genres and favourite_genres
# encoded_new_genres1 = encoded_new_genres[:, :len(new_df['genres'][0])]
# encoded_new_genres2 = encoded_new_genres[:, len(new_df['genres'][0]):]
#
# # Concatenate the one-hot encoded columns
# X_new = np.concatenate([encoded_new_genres1, encoded_new_genres2], axis=1)
#
# y_new_pred = model.predict(X_new)
# y_new_pred_classes = (y_new_pred > 0.5).astype(int)
#
# # Print the predicted labels
# print("Predicted labels:", y_new_pred_classes.flatten())
# print("Actual labels:", labels)
